# Exploration of Published Models

(discussion)

## Structural model

The structural model reported in [1] is a system of ordindary differential and algebraic equations. It combines a mechanistic description of the pharmacokinetics (PK) and the pharacodynamics (PD) of the compound, as well as an empirical model of the tumour growth.

The distribution of the compound in the mouse system after an oral dose is modelled by a one-compartment PK model with indirect drug administration

\begin{align*}
\frac{\text{d}A_d}{\text{d}t} = r_d(t) - k_{a}A_d,\qquad \frac{\text{d}A_c}{\text{d}t} = k_{a} A_d - k_e A_c,\qquad C_c = \frac{A_c}{V_c}.
\end{align*}

Here, $A_d$ is the amount of the compound in the dose compartment, and $A_c$ the amount in the central compartment. The dose compartment may be interpreted as the gut compartment. As the drug is orally administered the uptake into the dose compartment may be modelled by a time dependent dose rate $r_d$. The drug diffuses at a constant rate $k_a$ (absorption rate) into the central compartment. The central compartment models the drug circulating in the plasma. The clearance of the drug through the liver is modelled with a linear elimination rate $k_e$ from the central compartment. The concentration of the compound in the central compartment $C_c$ can be derived by normalising the amount of the drug by its effective volume of distribution in the central compartment $V_c$.

The tumour growth is modelled by an empirical model in [1]. In absence of the drug the tumour growth is modelled by an exponential growth phase for small tumour volumes, and by a linear growth phase for large tumour volumes. The growth inhibiting effects of the compound are assumed to be proportional to its concentration in central compartment and the volume of the tumour

\begin{align*}
\frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1} - \kappa C_cV^s_T.
\end{align*}

Here, $V^s_T$ is the predicted tumour volume by the structural model, $\lambda _0$ is the exponential growth rate, $\lambda _1$ is the linear growth rate, and $\kappa $ is the growth inhibiting potency of the compound.

In [1], the dose and central compartment were assumed to contain no amount of the compound initially, $A_d(t=0)=0$ and $A_c(t=0)=0$. As the dose rate $r_t$ is determined by the dosing regimen, this leaves the structural model with seven parameters that determine the PKPD of the drug

\begin{align*}
\psi = (k_a, k_e, V_c, V_0, \lambda _0, \lambda _1, \kappa ),
\end{align*}

where $V_0$ is the initial tumour volume, $V^s_T(t=0)=V_0$.

We have implemented the structural model in a separate python [module](https://github.com/DavAug/ErlotinibGefitinib/blob/master/pkpd/model.py), such that its myokit representation can be created with `pkpd.model.create_pktgi_model()`.

In [1]:
#
# Create structural model.
#

import pkpd.model as m


# Create model
model = m.create_pktgi_model()

# Set route of administration to 'oral dosing'
model.set_roa(dose_comp='central', indirect=True)

# Show model
print(model.code())

[[model]]
# Initial values
central.amount   = 0
central.volume_t = 0
depot.amount     = 0

[central]
dot(amount) = -k_e * amount + depot.k_a * depot.amount
    in [mg]
conc = amount / volume_c
    in [mg/L]
k_e = 0
    in [1/day]
kappa = 0
    in [L/mg/day]
lambda_0 = 0
    in [1/day]
lambda_1 = 1
    in [cm^3/day]
time = 0 bind time
    in [day]
volume_c = 1
    in [L]
dot(volume_t) = 2 * (lambda_0 * (lambda_1 * volume_t)) / (2 * (lambda_0 * volume_t) + lambda_1) - kappa * (conc * volume_t)
    in [cm^3]

[depot]
dot(amount) = -k_a * amount + dose_rate * regimen
    in [mg]
dose_rate = 0
    in [mg/day]
k_a = 0
    in [1/day]
regimen = 0 bind pace
    in [1]




## Error model

The expected random deviations of the observed tumour volumes from the structural model predictions $V^s_T$ are modelled in [1] by a combined error model. A combined error model assumes that the deviations $\varepsilon $ of the observable tumour volume $V_T$ from the structural model predictions $V^s_T$ may be modelled by a base-level Gaussian noise, and a Gaussian noise contribution whose standard deviation grows relative to the structural model predictions

\begin{equation*}
    \varepsilon = \left(\sigma _{\text{base}} + \sigma _{\text{rel}} V^s_T\right) \varepsilon _n.
\end{equation*}

Here, $\sigma _{\text{base}}$ is the standard deviation of the base-level noise, and $\sigma _{\text{rel}}$ is the standard deviation relative to $V^s_T$. $\varepsilon _n$ is a standard Gaussian random variable $\mathcal{N}(0, 1)$. As a result, the model predictions of the tumour volume are Gaussian-distributed, centered at the structural model predictions $V^s_T = V^s_T(t, r_d; \psi)$ and a standard deviation $\sigma _{\text{tot}} = \sigma _{\text{base}} + \sigma _{\text{rel}} V^s_T$

\begin{equation*}
    V_T \sim \mathcal{N}(V^s_T, \sigma ^2_{\text{tot}}).
\end{equation*}

The error model introduces two further parameters

\begin{equation*}
    \theta _V = (\sigma _{\text{base}}, \sigma _{\text{rel}}),
\end{equation*}

such that the PKPD model has nine parameters ($\psi$, $\theta _V$).

## Population model

In [1] it is assumed that some of the parameters $\psi $ can vary between individuals, while the error model parameters $\theta _V$ remain the same. In particular, all $\psi $ except the absorption rate $k_a$ are assumed to be log-normally distributed in the mouse population

\begin{equation*}
    \log \psi _i \sim \mathcal{N} (\mu _i, \sigma ^2_i), \quad \psi _i \neq k_a,
\end{equation*}

where $\mu _i $ is the mean, and $\sigma _i$ the standard deviation of $\log \psi _i$ in the population. As a result, for each $\psi _i$ except the absorption rate, the constant parameter is replaced by a distribution parameterised by the population mean and variance. Defining the population parameters as 

\begin{equation*}
    \theta _{\psi } = (k_a, \mu _{k_e}, \sigma _{k_e}, \mu _{V_c}, \sigma _{V_c}, \mu _{V_0}, \sigma _{V_0}, \mu _{\lambda _0}, \sigma _{\lambda _0}, \mu _{\lambda _1}, \sigma _{\lambda _1}, \mu _{\kappa}, \sigma _{\kappa})
\end{equation*}

The full population tumour growth inhibition PKPD of Erlotinib and Gefitinib is parameterised by 15 parameters each, $(\theta _{\psi}, \theta _V)$.

## Parameter estimates

The in [1] inferred model parameters $(\theta _{\psi}, \theta _V)$ are documented in Table 1, p. 3117 [1]. Note that the "Estimate" for a parameter $\psi _i$ corresponds in our notation to the exponential of the population mean, $\exp \mu _i$, while the interindividual variability "Var" is equivalent to the population standard deviation $\sigma _i$. The provided relative standard error (RSE) is the standard deviation of the estimates relative to their value. For later convenience we will translate the RSE values back into absolute standard deviations. For more details on the Monolix's conventions please have a look at its [documentation](http://monolix.lixoft.com/data-and-models/individualdistribution/).

In [32]:
#
# Estimates according to Eigenmann et. al. [1] for the Erlotinib and Gefitinib PKPD in LXF A677 and VXF A431.
#

import pandas as pd


# Create pandas dataframe with estimate, standard deviation entries LXF A677
lxf_erlotinib_estimates = pd.DataFrame(
    data={
        'k_a in 1/day': [55.0, None],
        'pop. mean V_c in L': [0.127, 0.127 * 0.15],
        'sigma V_c in "log L"': [0.251, None],
        'pop. mean k_e in 1/day': [7.56, 7.56 * 0.1],
        'sigma k_e in "log 1/day"': [0.332, 0.332 * 0.39],
        'pop. mean V_0 in mm^3': [122, 122 * 0.05],
        'sigma V_0 in "log mm^3"': [0.368, 0.122 * 0.05],
        'pop. mean lambda_0 in 1/day': [0.0971, 0.0971 * 0.08],
        'sigma lambda_0 in "log 1/day"': [0.456, 0.456 * 0.13],
        'pop. mean lambda_1 in mm^3/day': [127, 127 * 0.13],
        'pop. mean kappa in L/µg/day': [0.000117, 0.000117 * 0.17],
        'sigma kappa in "log L/µg/day"': [0.654, 0.654 * 0.21],
        'sigma lambda_1 in "log mm^3/day"': [0.710, None],
        'sigma base in mm^3': [14.1, 14.1 * 0.09],
        'sigma rel in dimless': [0.0907, 0.0907 * 0.06]},
    index=['estimate', 'standard deviation'])

# Show dataframe
print('Erlotinib LXF A677')
display(lxf_erlotinib_estimates)

# Create pandas dataframe with estimate, standard deviation entries VXF A341
vxf_erlotinib_estimates = pd.DataFrame(
    data={
        'k_a in 1/day': [55.0, None],
        'pop. mean V_c in L': [0.120, 0.120 * 0.06],
        'sigma V_c in "log L"': [0.190, 0.190 * 0.23],
        'pop. mean k_e in 1/day': [7.85, 7.85 * 0.07],
        'sigma k_e in "log 1/day"': [0.235, 0.235 * 0.26],
        'pop. mean V_0 in mm^3': [114, 114 * 0.06],
        'sigma V_0 in "log mm^3"': [0.423, 0.423 * 0.11],
        'pop. mean lambda_0 in 1/day': [0.0290, 0.0290 * 0.06],
        'sigma lambda_0 in "log 1/day"': [0.321, 0.321 * 0.14],
        'pop. mean lambda_1 in mm^3/day': [298, 298 * 0.37],
        'pop. mean kappa in L/µg/day': [0.0000401, 0.0000401 * 0.16],
        'sigma kappa in "log L/µg/day"': [0.585, 0.585 * 0.21],
        'sigma lambda_1 in "log mm^3/day"': [0.68, None],
        'sigma base in mm^3': [4.32, 4.32 * 0.3],
        'sigma rel in dimless': [0.139, 0.139 * 0.05]},
    index=['estimate', 'standard deviation'])

# Show dataframe
print('Erlotinib VXF A341')
display(vxf_erlotinib_estimates)

Erlotinib LXF A677


,k_a in 1/day,pop. mean V_c in L,"sigma V_c in ""log L""",pop. mean k_e in 1/day,"sigma k_e in ""log 1/day""",pop. mean V_0 in mm^3,"sigma V_0 in ""log mm^3""",pop. mean lambda_0 in 1/day,"sigma lambda_0 in ""log 1/day""",pop. mean lambda_1 in mm^3/day,pop. mean kappa in L/µg/day,"sigma kappa in ""log L/µg/day""","sigma lambda_1 in ""log mm^3/day""",sigma base in mm^3,sigma rel in dimless
estimate,55.0,0.12700,0.251,7.560,0.33200,122.0,0.3680,0.097100,0.45600,127.00,0.000117,0.65400,0.71,14.100,0.090700
standard deviation,NaN,0.01905,NaN,0.756,0.12948,6.1,0.0061,0.007768,0.05928,16.51,0.000020,0.13734,NaN,1.269,0.005442


Erlotinib VXF A341


,k_a in 1/day,pop. mean V_c in L,"sigma V_c in ""log L""",pop. mean k_e in 1/day,"sigma k_e in ""log 1/day""",pop. mean V_0 in mm^3,"sigma V_0 in ""log mm^3""",pop. mean lambda_0 in 1/day,"sigma lambda_0 in ""log 1/day""",pop. mean lambda_1 in mm^3/day,pop. mean kappa in L/µg/day,"sigma kappa in ""log L/µg/day""","sigma lambda_1 in ""log mm^3/day""",sigma base in mm^3,sigma rel in dimless
estimate,55.0,0.1200,0.1900,7.8500,0.2350,114.00,0.42300,0.02900,0.32100,298.00,0.000040,0.58500,0.68,4.320,0.13900
standard deviation,NaN,0.0072,0.0437,0.5495,0.0611,6.84,0.04653,0.00174,0.04494,110.26,0.000006,0.12285,NaN,1.296,0.00695


Note that the population mean of the parameters $\psi$ is not of the same scale as the standard deviation of the log-normal distribution $\sigma _i$. To illustrate this point, we plotted the inferred distribution of Erlotinib's potency across indiviudal mice for LXF A677 and VXF A431 below. The population means reported in [1] were 0.117 L/mg/day and 0.0401 L/mg/day with log-normal standard deviations of 0.654 and 0.585.

In [20]:
#
# Illustration of Erlotinib's potency population distribution for LXF A677 mice assuming estimate for population mean and standard deviation in [1] were exact.
#

import numpy as np
import plotly.graph_objects as go
from scipy.stats import lognorm


# Estimates for mean and s.d. kappa from Table 1, LXF A677
mean = 0.117  # in "log(L/mg/day)"
sd = 0.654  # in "log(L/mg/day)"

# Get 1st and 99th percentile
start = -2  
end = np.log10(0.4)

# Trace out probability density
lxf_kappa = np.logspace(start=start, stop=end)
lxf_pdf = lognorm.pdf(lxf_kappa, scale=mean, s=sd)

# Estimates for mean and s.d. kappa from Table 1, LXF A677
mean = 0.0401  # in "log(L/mg/day)"
sd = 0.585  # in "log(L/mg/day)"

# Get 1st and 99th percentile
start = -2  
end = np.log10(0.4)

# Trace out probability density
vxf_kappa = np.logspace(start=start, stop=end)
vxf_pdf = lognorm.pdf(vxf_kappa, scale=mean, s=sd)  # log(x/mean) = log(x) - log(mean)

# Create figure
fig = go.Figure()

# Plot dsitribution of kappa for LXF A677
fig.add_trace(go.Scatter(
    x=lxf_kappa,
    y=lxf_pdf,
    name='LXF A677',
    hovertemplate=
        "Potency of Erlotinib: %{x:.02f} L/mg/day<br>" +
        "Probability density in population: %{y:.02f}<br>" +
        "<extra></extra>",
    mode="lines",
))

# Plot distribution of kappa for VXF A431
fig.add_trace(go.Scatter(
    x=vxf_kappa,
    y=vxf_pdf,
    name='VXF A431',
    hovertemplate=
        "Potency of Erlotinib: %{x:.02f} L/mg/day<br>" +
        "Probability density in population: %{y:.02f}<br>" +
        "<extra></extra>",
    mode="lines",
))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=True,
    xaxis_title=r'$\text{Erlotinib potency }\kappa \text{ in L/mg/day}$',
    yaxis_title=r'$\text{Probability density in population}$',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"xaxis.type": "linear"}],
                    label="Linear x-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"xaxis.type": "log"}],
                    label="Log x-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Show figure
fig.show()

0.20876826722338207

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)